# **Download ultralytics**

In [1]:
!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 51802, done.
remote: Counting objects: 100% (944/944), done.
remote: Compressing objects: 100% (631/631), done.
remote: Total 51802 (delta 682), reused 317 (delta 313), pack-reused 50858 (from 4)
Receiving objects: 100% (51802/51802), 30.33 MiB | 25.05 MiB/s, done.
Resolving deltas: 100% (37980/37980), done.


# install in developer mode

In [2]:
%cd ultralytics
!pip install -e '.[dev]'

/kaggle/working/ultralytics
Obtaining file:///kaggle/working/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 40.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 94.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.3/380.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 89.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.6/451.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.5/264.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.5/128.5 kB 6.8 

# check machine configuration and H/W

In [4]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6170.3/8062.4 GB disk)


# Code to be added to blocks.py

In [ ]:
class ECAConv(nn.Module):
    """Efficient Channel Attention (ECA) module"""
    def __init__(self, c, b=1, gamma=2):
        super().__init__()
        k = int(abs((torch.log2(torch.tensor(c)) + b) / gamma))
        k = k if k % 2 else k + 1  # Ensure k is odd
        self.conv = nn.Conv1d(1, 1, kernel_size=k, padding=k // 2, bias=False)

    def forward(self, x):
        y = torch.mean(x, dim=(2, 3), keepdim=True)  # Global average pooling
        y = self.conv(y.squeeze(-1).transpose(-1, -2)).transpose(-1, -2).unsqueeze(-1)
        return x * torch.sigmoid(y)

class GSConv(nn.Module):
    """Ghost Shuffle Convolution for efficient feature extraction"""
    def __init__(self, c1, c2, k=3, s=1, p=None, g=1, d=1):
        super().__init__()
        c_ = c2 // 2  # Half channels
        self.primary_conv = nn.Conv2d(c1, c_, k, s, p, groups=g, dilation=d, bias=False)
        self.cheap_conv = nn.Conv2d(c_, c_, 1, 1, bias=False)  # Ghost Feature Generation

    def forward(self, x):
        y1 = self.primary_conv(x)
        y2 = self.cheap_conv(y1)
        return torch.cat([y1, y2], dim=1)  # Channel-wise concatenation

class EfficientConv(nn.Module):
    """Efficient Convolution block combining Conv2d, GSConv, and ECA attention"""
    def __init__(self, c1, c2, k=3, s=1, p=1, g=1, d=1, act=True, use_bn=True, attention=True):
        super().__init__()
        c_half = c2 // 2  # Split channels for Conv2d and GSConv
        self.conv = nn.Conv2d(c1, c_half, k, s, p, groups=g, dilation=d, bias=False)
        self.gsconv = GSConv(c1, c_half, k, s, p, g, d)  # Efficient convolution
        self.bn = nn.BatchNorm2d(c2) if use_bn else nn.Identity()
        self.act = nn.SiLU() if act else nn.Identity()
        self.eca = ECAConv(c2) if attention else nn.Identity()  # Efficient Channel Attention

    def forward(self, x):
        y1 = self.conv(x)
        y2 = self.gsconv(x)
        y = torch.cat([y1, y2], dim=1)  # Concatenation
        return self.act(self.eca(self.bn(y)))  # Final output

In [4]:
%%writefile /kaggle/working/ultralytics/ultralytics/nn/modules/block.py

Overwriting /kaggle/working/ultralytics/ultralytics/nn/modules/block.py


# update tasks.py

Overwriting /kaggle/working/ultralytics/ultralytics/nn/tasks.py


# update __init__.py

Overwriting /kaggle/working/ultralytics/ultralytics/nn/modules/__init__.py


## configure updated YOLOv8-model (CFG)

In [10]:
%%writefile /kaggle/working/ultralytics/ultralytics/cfg/models/v8/yolov8-mobile.yaml
# Ultralytics 🚀 AGPL-3.0 License - https://ultralytics.com/license

# Ultralytics YOLOv8 object detection model with P3/8 - P5/32 outputs
# Model docs: https://docs.ultralytics.com/models/yolov8
# Task docs: https://docs.ultralytics.com/tasks/detect

# Parameters
nc: 80 # number of classes
scales: # model compound scaling constants, i.e. 'model=yolov8n.yaml' will call yolov8.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024] # YOLOv8n summary: 129 layers, 3157200 parameters, 3157184 gradients, 8.9 GFLOPS
  s: [0.33, 0.50, 1024] # YOLOv8s summary: 129 layers, 11166560 parameters, 11166544 gradients, 28.8 GFLOPS
  m: [0.67, 0.75, 768] # YOLOv8m summary: 169 layers, 25902640 parameters, 25902624 gradients, 79.3 GFLOPS
  l: [1.00, 1.00, 512] # YOLOv8l summary: 209 layers, 43691520 parameters, 43691504 gradients, 165.7 GFLOPS
  x: [1.00, 1.25, 512] # YOLOv8x summary: 209 layers, 68229648 parameters, 68229632 gradients, 258.5 GFLOPS

# YOLOv8.0n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, EfficientConv, [64, 3, 2]] # 0-P1/2
  - [-1, 1, EfficientConv, [128, 3, 2]] # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, EfficientConv, [256, 3, 2]] # 3-P3/8
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, EfficientConv, [512, 3, 2]] # 5-P4/16
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, EfficientConv, [1024, 3, 2]] # 7-P5/32
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]] # 9

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 3, C2f, [512]] # 12

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 3, C2f, [256]] # 15 (P3/8-small)

  - [-1, 1, EfficientConv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]] # cat head P4
  - [-1, 3, C2f, [512]] # 18 (P4/16-medium)

  - [-1, 1, EfficientConv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]] # cat head P5
  - [-1, 3, C2f, [1024]] # 21 (P5/32-large)

  - [[15, 18, 21], 1, Detect, [nc]] # Detect(P3, P4, P5)


Writing /kaggle/working/ultralytics/ultralytics/cfg/models/v8/yolov8-mobile.yaml


# create a new file (lets name it train.py), and check model configuration

In [11]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO

if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v8/yolov8-mobile.yaml')
    model.info()

Writing train.py


In [12]:
!python train.py

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
WARNING ⚠️ no model scale passed. Assuming scale='n'.
YOLOv8-mobile summary: 157 layers, 3,019,855 parameters, 3,019,839 gradients, 8.6 GFLOPs


# Everything is file and ready to go

Let's download the dataset from Roboflow

In [18]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="y4uzPDgeB6rEDxdHhxsW")
project = rf.workspace("buddetection").project("bud_detection-np")
version = project.version(1)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.5 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to bud_detection-np-1 in yolov8:: 100%|██████████| 6468/6468 [00:00<00:00, 6828.14it/s]


# update data.yaml file

In [19]:
%%writefile /kaggle/working/ultralytics/bud_detection-np-1/data.yaml
names:
- BUD
nc: 1
roboflow:
  license: CC BY 4.0
  project: bud_detection-np
  url: https://universe.roboflow.com/buddetection/bud_detection-np/dataset/1
  version: 1
  workspace: buddetection
test: /kaggle/working/ultralytics/bud_detection-np-1/test/images
train: /kaggle/working/ultralytics/bud_detection-np-1/train/images
val: /kaggle/working/ultralytics/bud_detection-np-1/valid/images

Overwriting /kaggle/working/ultralytics/bud_detection-np-1/data.yaml


# update the train.py to enable the training

In [20]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v8/yolov8-mobile.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/bud_detection-np-1/data.yaml", epochs=100, batch=16, imgsz=640)
    #model.train(resume=True)

Overwriting train.py


# !!!START THE TRAINING!!!

In [21]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.90 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-mobile.yaml, data=/kaggle/working/ultralytics/bud_detection-np-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=20, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fa

# RESULTS ARE HERE!!!

Download the results

In [22]:
!zip -r file.zip runs

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/train/ (stored 0%)
  adding: runs/detect/train/results.csv (deflated 61%)
  adding: runs/detect/train/val_batch1_pred.jpg (deflated 5%)
  adding: runs/detect/train/val_batch1_labels.jpg (deflated 5%)
  adding: runs/detect/train/labels.jpg (deflated 24%)
  adding: runs/detect/train/train_batch1.jpg (deflated 1%)
  adding: runs/detect/train/labels_correlogram.jpg (deflated 35%)
  adding: runs/detect/train/args.yaml (deflated 53%)
  adding: runs/detect/train/train_batch14801.jpg (deflated 5%)
  adding: runs/detect/train/val_batch2_pred.jpg (deflated 6%)
  adding: runs/detect/train/events.out.tfevents.1742012830.e25b78843490.119.0 (deflated 88%)
  adding: runs/detect/train/P_curve.png (deflated 17%)
  adding: runs/detect/train/val_batch0_labels.jpg (deflated 5%)
  adding: runs/detect/train/train_batch0.jpg (deflated 1%)
  adding: runs/detect/train/results.png (deflated 9%)
  adding: runs/detect/train/train

# Below YOLOv10n results are collected on sugarcane bud detection dataset

In [23]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v10/yolov10n.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/bud_detection-np-1/data.yaml", epochs=100, batch=32, imgsz=640)
    #model.train(resume=True)

Overwriting train.py


In [24]:
!python train.py

Ultralytics 8.3.90 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v10/yolov10n.yaml, data=/kaggle/working/ultralytics/bud_detection-np-1/data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False,

# This is a proposed model training on VisDrone Dataset 

In [13]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v8/yolov8-mobile.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="VisDrone.yaml", epochs=50, batch=32, imgsz=640)
    #model.train(resume=True)

Overwriting train.py


In [14]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.90 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-mobile.yaml, data=VisDrone.yaml, epochs=50, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sav